In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import diagram
from spytial.annotations import orientation, attribute, hideAtom, atomColor, group, flag
from spytial import relationalizer, RelationalizerBase, Atom, Relation

import dd
print(dd.__version__)

0.6.0


## DD is a standard Python lib for BDDs

- But no clear obv visualizer beyond dumping to network X and dot graphs.
- https://github.com/tulip-control/dd?tab=readme-ov-file



In [2]:
from dd.autoref import BDD

bdd = BDD()
bdd.declare('x', 'y', 'z')
u = bdd.add_expr(r'(x /\ y) \/ ~z')

# Map variable indices to names for readability
var_names = {idx: name for name, idx in bdd.vars.items()}

print("BDD node structure (node_id: var_name, low ->, high ->):")
for node, (var, low, high) in bdd._bdd._succ.items():
    var_label = var_names.get(var, str(var))
    print(f"Node {node}: var={var_label}, low={low}, high={high}")

#bdd.dump('bdd.dot')


# # Dump to JSON
# filename = 'bdd.json'
# roots = dict(u=u)
# bdd.dump(filename, roots)

BDD node structure (node_id: var_name, low ->, high ->):
Node 1: var=3, low=None, high=None
Node 2: var=x, low=-1, high=1
Node 3: var=y, low=-1, high=1
Node 4: var=x, low=-1, high=3
Node 5: var=z, low=-1, high=1
Node 6: var=y, low=-5, high=1
Node 7: var=x, low=-5, high=6


In [ ]:
@relationalizer(priority=101)
class BDDRelationalizer(RelationalizerBase):
    def can_handle(self, obj):
        # Only handle dd.autoref.BDD objects
        return isinstance(obj, BDD)

    def relationalize(self, obj, walker_func):
        atoms = []
        relations = []
        # Map variable indices to names
        var_names = {idx: name for name, idx in obj.vars.items()}
        seen = set()
        false_id = obj.false.node
        true_id = obj.true.node

        # Add all decision (non-terminal) nodes
        for node, (var, low, high) in obj._bdd._succ.items():
            label = var_names.get(var, f"var_{var}")
            atoms.append(Atom(id=str(node), type='BDDNode', label=label))
            seen.add(node)
            if low is not None:
                relations.append(Relation(name='low', atoms=[str(node), str(low)]))
            if high is not None:
                relations.append(Relation(name='high', atoms=[str(node), str(high)]))

        # Add atoms for all referenced terminal and complemented nodes
        for rel in relations:
            for target in rel.atoms[1:]:
                tid = int(target)
                if tid not in seen:
                    if tid == false_id or target == "F":
                        atoms.append(Atom(id=str(tid), type='Terminal', label='False'))
                        seen.add(tid)
                    elif tid == true_id or target == "T":
                        atoms.append(Atom(id=str(tid), type='Terminal', label='True'))
                        seen.add(tid)
                    elif tid < 0:
                        pos_tid = abs(tid)
                        # If the positive node exists, use its variable name
                        if pos_tid in obj._bdd._succ:
                            var_idx = obj._bdd._succ[pos_tid][0]
                            var_label = var_names.get(var_idx, f"var_{var_idx}")
                            atoms.append(Atom(id=str(tid), type='Complemented', label=f'NOT({var_label})'))
                        elif pos_tid == false_id:
                            atoms.append(Atom(id=str(tid), type='Complemented', label='NOT(False)'))
                        elif pos_tid == true_id:
                            atoms.append(Atom(id=str(tid), type='Complemented', label='NOT(True)'))
                        else:
                            atoms.append(Atom(id=str(tid), type='Complemented', label=f'NOT({pos_tid})'))
                        seen.add(tid)
        return atoms, relations

# # Dump to JSON
filename = 'bdd.json'
roots = dict(u=u)
bdd.dump(filename, roots)

diagram(bdd)